In [ ]:
import os
import gym
import numpy as np
import cv2
from gym.spaces import Box
from nes_py.wrappers import JoypadSpace
import gym_super_mario_bros
from gym_super_mario_bros.actions import SIMPLE_MOVEMENT
from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import DummyVecEnv, VecFrameStack
from stable_baselines3.common.callbacks import CheckpointCallback
from stable_baselines3.common.utils import set_random_seed

# 1. CARTELLE
CHECKPOINT_DIR = './train/nmod'
LOG_DIR = './logs/'
os.makedirs(CHECKPOINT_DIR, exist_ok=True)
os.makedirs(LOG_DIR, exist_ok=True)

class MarioModernized(gym.Env):
    def __init__(self):
        super().__init__()
        inner_env = gym_super_mario_bros.make('SuperMarioBros-1-1-v0')
        self.mario = JoypadSpace(inner_env, SIMPLE_MOVEMENT)
        self.observation_space = Box(low=0, high=255, shape=(84, 84, 1), dtype=np.uint8)
        self.action_space = self.mario.action_space
        
        # Variabili di controllo
        self.max_x = 0
        self.stagnant_steps = 0
        self.skip_frames = 4 

    # Aggiungi questo nell' __init__ della classe MarioModernized
    # self.frame_stack_view = [] 

    def _process_frame(self, frame):
        if frame is not None:
            # 1. Elaborazione standard per l'IA
            gray = cv2.cvtColor(frame, cv2.COLOR_RGB2GRAY)
            resized = cv2.resize(gray, (84, 84), interpolation=cv2.INTER_AREA)
            
            # 2. Logica per la "Striscia di Pellicola" (Visualizzazione per te)
            # Creiamo una lista di immagini per mostrare la "scia"
            if not hasattr(self, 'view_stack'):
                self.view_stack = [np.zeros((84, 84), dtype=np.uint8)] * 4
            
            # Aggiorniamo la scia: togliamo il più vecchio, aggiungiamo il nuovo
            self.view_stack.pop(0)
            self.view_stack.append(resized)
            
            # Affianchiamo i 4 frame orizzontalmente (np.hstack)
            film_strip = np.hstack(self.view_stack)
            
            # Ingrandiamo la striscia per vederla bene (es. 4 volte più grande)
            big_strip = cv2.resize(film_strip, (84*4*2, 84*2), interpolation=cv2.INTER_NEAREST)
            
            # Aggiungiamo dei bordi bianchi tra i frame per distinguerli
            cv2.imshow("Memoria IA (4 Frame consecutivi)", big_strip)
            cv2.waitKey(1)
            
            return resized[:, :, np.newaxis]
        return np.zeros((84, 84, 1), dtype=np.uint8)

    def reset(self, seed=None, options=None):
        self.max_x = 0
        self.stagnant_steps = 0
        obs = self.mario.reset()
        self.mario.render()
        return self._process_frame(obs)

    

    def step(self, action):
        total_reward = 0
        last_x = self.max_x
        
        for _ in range(self.skip_frames):
            obs, reward, done, info = self.mario.step(action)
            total_reward += reward
            if done: break

        self.mario.render()
        
        # --- LOGICA PREMI CORRETTA (REWARD SHAPING) ---
        current_x = info['x_pos']
        
        # 1. PREMIO PER AVANZAMENTO (Delta X)
        # Premiamo Mario solo se supera il suo record personale di distanza
        if current_x > self.max_x:
            diff = current_x - self.max_x
            total_reward += diff  # Riceve punti per ogni pixel nuovo scoperto
            self.max_x = current_x
            self.stagnant_steps = 0
        else:
            self.stagnant_steps += 1

        # 2. PUNIZIONE MORTE
        if done and info['flag_get'] is False:
            total_reward -= 50  # Morire è molto male

        # 3. BONUS BANDIERA
        if info['flag_get']:
            print("!!! BANDIERA RAGGIUNTA !!!")
            total_reward += 500
            
        # 4. PUNIZIONE PIGRIZIA (Stagnamento)
        # Se non avanza per troppo tempo, punizione e reset
        if self.stagnant_steps > 150: # Circa 10 secondi reali con frame skipping
            total_reward -= 20
            done = True 

        obs = self._process_frame(obs)
        # Normalizziamo leggermente il reward per stabilità
        return obs, float(total_reward / 10.0), done, info

if __name__ == "__main__":
    SEED = 42
    set_random_seed(SEED)
    
    env = DummyVecEnv([lambda: MarioModernized()])
    env.seed(SEED)
    env = VecFrameStack(env, n_stack=4, channels_order='last')

    checkpoint_callback = CheckpointCallback(
        save_freq=100000, 
        save_path=CHECKPOINT_DIR,
        name_prefix='mario_model'
    )

    # PARAMETRI PPO OTTIMIZZATI
    model = PPO(
        "CnnPolicy", 
        env, 
        verbose=1, 
        learning_rate=0.0001,
        n_steps=2048,      # Aumentato: raccoglie più dati prima di aggiornare (molto più stabile)
        batch_size=128,    # Aumentato: rende il gradiente più preciso
        n_epochs=10,       # Quante volte analizza lo stesso blocco di dati
        ent_coef=0.01,     # Ridotto leggermente per evitare caos eccessivo
        tensorboard_log=LOG_DIR,
        seed=SEED
    )

    print("Training avviato. Ora Mario deve avanzare per ricevere punti.")
    try:
        model.learn(total_timesteps=1000000, callback=checkpoint_callback)
    except KeyboardInterrupt:
        model.save("mario_model_final")

Using cuda device
Wrapping the env in a VecTransposeImage.
Training avviato. Ora Mario deve avanzare per ricevere punti.
Logging to ./logs/PPO_9


C:\Users\matte\Desktop\marioia\mario_stabile\Lib\site-packages\gym_super_mario_bros\smb_env.py:148: RuntimeWarning: overflow encountered in scalar subtract
  return (self.ram[0x86] - self.ram[0x071c]) % 256


-----------------------------
| time/              |      |
|    fps             | 54   |
|    iterations      | 1    |
|    time_elapsed    | 37   |
|    total_timesteps | 2048 |
-----------------------------
